## Extraccion, Tranformacion y Carga

In [1]:
import json                              # importo modulo json encoder and decoder
import pandas as pd                      # importo modulo panda Data Analysis Library
import ast                               # importo modulo Abstract Syntax Trees
from textblob import TextBlob            # importo modulo textblob 
import re                                # importo modulo re - regular expresion


Una vez importados los modulos, importo el dataset e inicio mi trabajo con pandas

      Incorporo el archivo australian_user_reviews

In [2]:
reviews = []

with open('Data/australian_user_reviews.json', 'r', encoding='utf-8') as f:
    # utilizo un bucle for para insertar datos en la lista
    for line in f.readlines():
        reviews.append(ast.literal_eval(line))
# creo el dataframe con los datos de la lista reviews
df_user_reviews = pd.DataFrame(reviews)
# muestro el dataframe
df_user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Muestro informacion del dataFrame

In [3]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


Verifico si existen duplicados

In [4]:
def verifica_duplicados_por_columna(df, columna):
    # Se filtran las filas duplicadas
    duplicated_rows = df[df.duplicated(subset=columna, keep=False)]
    if duplicated_rows.empty:
        return "No hay duplicados"
    
    # se ordenan las filas duplicadas para comparar entre sí
    duplicated_rows_sorted = duplicated_rows.sort_values(by=columna)
    return duplicated_rows_sorted

filas_duplicadas = verifica_duplicados_por_columna(df_user_reviews, 'user_id')
filas_duplicadas

,user_id,user_url,reviews
12888,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
5250,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
3133,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
3134,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
4139,29123,http://steamcommunity.com/id/29123,"[{'funny': '', 'posted': 'Posted March 26.', '..."
...,...,...,...
2721,xXAussieRockXx,http://steamcommunity.com/id/xXAussieRockXx,"[{'funny': '', 'posted': 'Posted July 17, 2015..."
2680,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
5855,zeroblade,http://steamcommunity.com/id/zeroblade,"[{'funny': '', 'posted': 'Posted November 30, ..."


Se observan 623 filas duplicadas en la columna 'User_id', pero se revisan si los review dentro de los datos anidados de 'Review' la información se encuentra duplicada o si solo se duplica el 'user_id' porque hay mas de un comentario realizado por ese usuario.

In [5]:
user_id = '05041129'
user_reviews = filas_duplicadas[filas_duplicadas['user_id'] == user_id]['reviews']

for review_list in user_reviews:
    for review in review_list:
        print(review['review'])
    print('-' * 40)

This game to me it is so good that it is better than any of the games out their and $15 worth it
this is the best third person game ever that i have played
this will be the  number one game if it have more competitive things
----------------------------------------
This game to me it is so good that it is better than any of the games out their and $15 worth it
this is the best third person game ever that i have played
this will be the  number one game if it have more competitive things
----------------------------------------


Se puede ver que los review son exactamente los mismos para cada registro, por lo que se decide borrar los duplicados, dejando la primer ocurrencia de los registros.

In [6]:
df_user_reviews = df_user_reviews.drop_duplicates(subset='user_id', keep='first')
def verifica_duplicados_por_columna(df, columna):
    # Se filtran las filas duplicadas
    duplicated_rows = df[df.duplicated(subset=columna, keep=False)]
    if duplicated_rows.empty:
        return "No hay duplicados"
    
    # se ordenan las filas duplicadas para comparar entre sí
    duplicated_rows_sorted = duplicated_rows.sort_values(by=columna)
    return duplicated_rows_sorted

filas_duplicadas = verifica_duplicados_por_columna(df_user_reviews, 'user_id')
filas_duplicadas

'No hay duplicados'

Verifico el contenido de la columna 'Reviews'

In [7]:
print(df_user_reviews['reviews'][0])

[{'funny': '', 'posted': 'Posted November 5, 2011.', 'last_edited': '', 'item_id': '1250', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'}, {'funny': '', 'posted': 'Posted July 15, 2011.', 'last_edited': '', 'item_id': '22200', 'helpful': 'No ratings yet', 'recommend': True, 'review': "It's unique and worth a playthrough."}, {'funny': '', 'posted': 'Posted April 21, 2011.', 'last_edited': '', 'item_id': '43110', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]


La columna 'Reviews' tiene una lista con mas de un diccionario. Dentro de los diccionarios se distinguen las siguientes claves: 

    Info , check Json keys
        {'funny': '',
        'posted': 'Posted November 5, 2011.',
        'last_edited': '',
        'item_id': '1250',
        'helpful': 'No ratings yet',
        'recommend': True,
        'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'}    

La columna 'Reviews' se encuentra anidada, por lo que se normaliza esta columna para obtener una clumna por cada clave encontrada en los diccionarios:

In [8]:
df_reviews = pd.json_normalize(df_user_reviews['reviews'])
df_reviews.head()

,0,1,2,3,4,5,6,7,8,9
0,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


En esta transformación, se pierde el 'user_id' y 'user_url' al que pertenece cada diccionario, pero aún mantiene la misma posición. Para volver a tener la trazabilidad del usuario, se concatena con el dataframe anterior.

In [9]:
df_reviews2 = pd.concat([df_user_reviews[['user_id', 'user_url']], df_reviews], axis=1)
df_reviews2.head()

,user_id,user_url,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,doctr,http://steamcommunity.com/id/doctr,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,maplemage,http://steamcommunity.com/id/maplemage,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


Ahora que se tienen los diccionarios por columnas, con el usuario que genera dicha información, se genera un registro por cada diccionario, manteniendo en cada caso el usuario que lo genera.

In [10]:
df_reviews2 = pd.melt(df_reviews2, id_vars=['user_id', 'user_url'], 
                       value_vars=list(range(9)),
                       value_name='reviews')
df_reviews2.head()

,user_id,user_url,variable,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,js41637,http://steamcommunity.com/id/js41637,0,"{'funny': '', 'posted': 'Posted June 24, 2014...."
2,evcentric,http://steamcommunity.com/id/evcentric,0,"{'funny': '', 'posted': 'Posted February 3.', ..."
3,doctr,http://steamcommunity.com/id/doctr,0,"{'funny': '', 'posted': 'Posted October 14, 20..."
4,maplemage,http://steamcommunity.com/id/maplemage,0,"{'funny': '3 people found this review funny', ..."


Al hacer esto último se puede ver que quedan registros None. Esto ocurre porque hay usuarios que hicieron mas reviews que otros. En este ejemplo se puede ver este caso:

In [11]:
df_reviews2[df_reviews2['user_id']=='76561197970982479']

,user_id,user_url,variable,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
25799,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,"{'funny': '', 'posted': 'Posted July 15, 2011...."
51598,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,"{'funny': '', 'posted': 'Posted April 21, 2011..."
77397,76561197970982479,http://steamcommunity.com/profiles/76561197970...,3,None
103196,76561197970982479,http://steamcommunity.com/profiles/76561197970...,4,None
128995,76561197970982479,http://steamcommunity.com/profiles/76561197970...,5,None
154794,76561197970982479,http://steamcommunity.com/profiles/76561197970...,6,None
180593,76561197970982479,http://steamcommunity.com/profiles/76561197970...,7,None
206392,76561197970982479,http://steamcommunity.com/profiles/76561197970...,8,None


Se eliminan los registros que tienen None en 'reviews'.

In [12]:
df_reviews2 = df_reviews2.dropna()
# Se verifica que solo queden el 'user_id' con la cantidad de diccionarios que le corresponde
df_reviews2[df_reviews2['user_id']=='76561197970982479']

,user_id,user_url,variable,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
25799,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,"{'funny': '', 'posted': 'Posted July 15, 2011...."
51598,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,"{'funny': '', 'posted': 'Posted April 21, 2011..."


Se separan por columnas cada una de las claves de 'Reviews'

In [13]:
df_reviews = df_reviews2['reviews'].apply(pd.Series, dtype='object')
df_reviews = df_reviews.add_prefix('reviews_')
df_reviews.head()

,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


En el procesamiento anterior, se puede ver que la columna de 'User_id' y 'User_url' se perdió nuevamente, por lo que se vuelve a concatenar.

In [14]:
df_reviews = pd.concat([df_reviews2[['user_id', 'user_url']], df_reviews], axis=1)
df_reviews.head()

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


Se observa que hay valores faltantes en algunas columnas, pero no estan como nulos, probablemente deben tener un espacio. Se compueba esto.

In [15]:
df_reviews['reviews_last_edited'][0]

''

Se reemplazan esos valores faltantes

In [16]:
df_reviews.replace('', None, inplace=True)
df_reviews.head()

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted November 5, 2011.",None,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,None,"Posted June 24, 2014.",None,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,None,Posted February 3.,None,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,None,"Posted October 14, 2013.",None,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",None,211420,35 of 43 people (81%) found this review helpful,True,Git gud


Muestro nuevamente informacion del dataframe resultante

In [17]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57397 entries, 0 to 231501
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   user_id              57397 non-null  object
 1   user_url             57397 non-null  object
 2   reviews_funny        7899 non-null   object
 3   reviews_posted       57397 non-null  object
 4   reviews_last_edited  5898 non-null   object
 5   reviews_item_id      57397 non-null  object
 6   reviews_helpful      57397 non-null  object
 7   reviews_recommend    57397 non-null  bool  
 8   reviews_review       57367 non-null  object
dtypes: bool(1), object(8)
memory usage: 6.0+ MB


Se revisa los tipos de datos de cada columna, a su vez se verifica la existencia de nulos en cada una de ellas.

In [18]:
def obtener_informacion_columnas(df):
    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}

    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].dtype)
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100 - porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())

    df_info = pd.DataFrame(mi_dict)
    return df_info
informacion_columnas = obtener_informacion_columnas(df_reviews)
print(informacion_columnas)


          nombre_campo tipo_datos  no_nulos_%  nulos_%  nulos
0              user_id     object      100.00     0.00      0
1             user_url     object      100.00     0.00      0
2        reviews_funny     object       13.76    86.24  49498
3       reviews_posted     object      100.00     0.00      0
4  reviews_last_edited     object       10.28    89.72  51499
5      reviews_item_id     object      100.00     0.00      0
6      reviews_helpful     object      100.00     0.00      0
7    reviews_recommend       bool      100.00     0.00      0
8       reviews_review     object       99.95     0.05     30


Se observa mas de un 85% de faltantes de datos en las columnas 'Reviews_funny','Reviews_last_edited', las cuales se eliminaran , ya que no se consideran necesarias para el analisis.

In [19]:
df_reviews.drop(['reviews_funny', 'reviews_last_edited'], axis=1, inplace=True)
df_reviews.columns


Index(['user_id', 'user_url', 'reviews_posted', 'reviews_item_id',
       'reviews_helpful', 'reviews_recommend', 'reviews_review'],
      dtype='object')

Se necesita que la fecha donde se hizo el posteo de la review este en formato `YYYY-MM-DD`, pero se encuentra como `Posted November 5, 2011.`. Por lo tanto, es necesario procesar la fecha y extraer los elementos relevantes.

In [20]:
def convertir_fecha(cadena_fecha):

    match = re.search(r'(\w+\s\d{1,2},\s\d{4})', cadena_fecha)
    if match:
        fecha_str = match.group(1)
        try:
            fecha_dt = pd.to_datetime(fecha_str)
            return fecha_dt.strftime('%Y-%m-%d')
        except:
            return 'Fecha inválida'
    else:
        return 'Formato inválido'
df_reviews['reviews_date'] = df_reviews['reviews_posted'].apply(convertir_fecha)
df_reviews['reviews_date']

0               2011-11-05
1               2014-06-24
2         Formato inválido
3               2013-10-14
4               2014-04-15
                ...       
231291          2014-08-15
231293          2014-08-02
231419          2015-07-31
231499          2015-12-20
231501    Formato inválido
Name: reviews_date, Length: 57397, dtype: object

Se puede observar que hay 9771 registros que contienen un formato inválido distinto a los demas registros. Estos registros no se podrán consultar desde la API, pero las demás columnas serán útiles para aportar información.

In [21]:
df_reviews[df_reviews['reviews_date'] == 'Formato inválido']

,user_id,user_url,reviews_posted,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,reviews_date
2,evcentric,http://steamcommunity.com/id/evcentric,Posted February 3.,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,Formato inválido
6,76561198079601835,http://steamcommunity.com/profiles/76561198079...,Posted May 20.,730,0 of 1 people (0%) found this review helpful,True,ZIKA DO BAILE,Formato inválido
7,MeaTCompany,http://steamcommunity.com/id/MeaTCompany,Posted July 24.,730,No ratings yet,True,BEST GAME IN THE BLOODY WORLD,Formato inválido
9,76561198156664158,http://steamcommunity.com/profiles/76561198156...,Posted June 16.,252950,0 of 1 people (0%) found this review helpful,True,love it,Formato inválido
10,76561198077246154,http://steamcommunity.com/profiles/76561198077...,Posted June 11.,440,No ratings yet,True,mt bom,Formato inválido
...,...,...,...,...,...,...,...,...
223569,76561198040184950,http://steamcommunity.com/profiles/76561198040...,Posted April 12.,394690,No ratings yet,True,I cannot say much right now due to the game no...,Formato inválido
226105,76561198046474248,http://steamcommunity.com/profiles/76561198046...,Posted March 28.,234140,No ratings yet,True,"Oh what a day .., What a lovely day to play th...",Formato inválido
228109,dmitry_who,http://steamcommunity.com/id/dmitry_who,Posted May 17.,376210,10 of 28 people (36%) found this review helpful,True,░░░░░░░░░░░█▀▀░░█░░░░░░░░░░░▄▀▀▀▀░░░░░█▄▄░░░░░...,Formato inválido
229231,76561198079507136,http://steamcommunity.com/profiles/76561198079...,Posted January 3.,730,No ratings yet,False,got VACed,Formato inválido


Se elimina la columna 'Review_posted' porque no aportan mayor informacion para el analisis

In [22]:
df_reviews = df_reviews.drop('reviews_posted', axis=1)
df_reviews.columns

Index(['user_id', 'user_url', 'reviews_item_id', 'reviews_helpful',
       'reviews_recommend', 'reviews_review', 'reviews_date'],
      dtype='object')

Se observan 5% de valores nulos en la columna 'Review_review'y se procede a eliminarlos

In [23]:
columnas_a_considerar = ['reviews_review']
df_reviews = df_reviews.dropna(subset=columnas_a_considerar)

Se analizan los tipos de datos y los nulos que quedaron finalmente en el dataframe.

In [24]:
def obtener_informacion_columnas(df):
    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}

    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].dtype)
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100 - porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())

    df_info = pd.DataFrame(mi_dict)
    return df_info
informacion_columnas = obtener_informacion_columnas(df_reviews)
print(informacion_columnas)

        nombre_campo tipo_datos  no_nulos_%  nulos_%  nulos
0            user_id     object       100.0      0.0      0
1           user_url     object       100.0      0.0      0
2    reviews_item_id     object       100.0      0.0      0
3    reviews_helpful     object       100.0      0.0      0
4  reviews_recommend       bool       100.0      0.0      0
5     reviews_review     object       100.0      0.0      0
6       reviews_date     object       100.0      0.0      0


# Analisis de Sentimientos

Crear la columna ***'sentiment_analysis'*** aplicando análisis de sentimiento con NLP con la siguiente escala: 
        
        Malo: '0' 
        Neutral: '1' 
        Poitivo: '2'
         
Esta nueva columna debe reemplazar la de review para facilitar el trabajo de los modelos de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de `1`

In [25]:
def analisis_sentimiento(review):
    if review is None:
        return 1
    analysis = TextBlob(review)
    polarity = analysis.sentiment.polarity
    if polarity < -0.2:
        return 0  
    elif polarity > 0.2: 
        return 2 
    else:
        return 1 
df_reviews['sentiment_analysis'] = df_reviews['reviews_review'].apply(analisis_sentimiento)
df_reviews.head()


,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,reviews_date,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011-11-05,1
1,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014-06-24,1
2,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,Formato inválido,2
3,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2013-10-14,2
4,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,Git gud,2014-04-15,1


Verifico valores de sentimiento en el dataframe

In [26]:
df_reviews.loc[df_reviews['sentiment_analysis'] == 0]

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,reviews_date,sentiment_analysis
5,Wackky,http://steamcommunity.com/id/Wackky,249130,7 of 8 people (88%) found this review helpful,True,This game is Marvellous.,2014-05-05,0
8,76561198089393905,http://steamcommunity.com/profiles/76561198089...,72850,3 of 3 people (100%) found this review helpful,True,"Killed the Emperor, nobody cared and got away ...",2015-02-01,0
16,76561198043472122,http://steamcommunity.com/profiles/76561198043...,33440,1 of 3 people (33%) found this review helpful,False,This Game Doesn't Work,2014-12-19,0
23,76561198048353577,http://steamcommunity.com/profiles/76561198048...,368230,3 of 5 people (60%) found this review helpful,True,♥♥♥♥ me where to start..-Made Mad Base-Got Red...,2015-10-26,0
24,76561198066046412,http://steamcommunity.com/profiles/76561198066...,359320,4 of 5 people (80%) found this review helpful,False,"♥♥♥♥♥ charged me 80 now its 15 dollars, got bo...",2015-12-28,0
...,...,...,...,...,...,...,...,...
229822,76561198085452317,http://steamcommunity.com/profiles/76561198085...,8980,0 of 4 people (0%) found this review helpful,True,A Gem.It's level of stupidity is just overwhem...,2013-12-23,0
230043,76561198087980305,http://steamcommunity.com/profiles/76561198087...,316790,No ratings yet,True,"Puedo decir ahora, que después de terminar el ...",2015-11-01,0
230571,76561198093736412,http://steamcommunity.com/profiles/76561198093...,219740,No ratings yet,True,"You will die, thats all you need to know is yo...",2014-10-19,0
230663,76561198095035915,http://steamcommunity.com/profiles/76561198095...,440,No ratings yet,True,Duh...,2013-08-11,0


In [27]:
df_reviews.loc[df_reviews['user_id'] == 'js41637']


,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,reviews_date,sentiment_analysis
1,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014-06-24,1
25800,js41637,http://steamcommunity.com/id/js41637,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2013-09-08,1
51599,js41637,http://steamcommunity.com/id/js41637,239030,1 of 4 people (25%) found this review helpful,True,Very fun little game to play when your bored o...,2013-11-29,1


In [28]:
df_reviews

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,reviews_date,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011-11-05,1
1,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014-06-24,1
2,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,Formato inválido,2
3,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2013-10-14,2
4,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,Git gud,2014-04-15,1
...,...,...,...,...,...,...,...,...
231291,How51,http://steamcommunity.com/id/How51,440,No ratings yet,True,TF2 is alot of fun and its really good but the...,2014-08-15,1
231293,76561198111410893,http://steamcommunity.com/profiles/76561198111...,304930,No ratings yet,True,Fun game with friends,2014-08-02,1
231419,zaza147,http://steamcommunity.com/id/zaza147,265630,No ratings yet,True,So Fun!! :D,2015-07-31,2
231499,lifeonhigh,http://steamcommunity.com/id/lifeonhigh,304050,No ratings yet,True,"This game is great. The only thing is,Why cant...",2015-12-20,1


In [29]:
def obtener_informacion_columnas(df):
    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}

    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].dtype)
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100 - porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())

    df_info = pd.DataFrame(mi_dict)
    return df_info
informacion_columnas = obtener_informacion_columnas(df_reviews)
print(informacion_columnas)

         nombre_campo tipo_datos  no_nulos_%  nulos_%  nulos
0             user_id     object       100.0      0.0      0
1            user_url     object       100.0      0.0      0
2     reviews_item_id     object       100.0      0.0      0
3     reviews_helpful     object       100.0      0.0      0
4   reviews_recommend       bool       100.0      0.0      0
5      reviews_review     object       100.0      0.0      0
6        reviews_date     object       100.0      0.0      0
7  sentiment_analysis      int64       100.0      0.0      0


## Carga de datos

Una vez realizada la limpieza de datos, se cargan los mismos en un nuevo archivo `df_user_reviews.csv`

In [30]:
df_reviews.to_csv('Datos/df_user_reviews.csv', index=False)